Copyright 2016 Google Inc. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

--------------------------------------

This notebook is similar in functionality to [this python script](https://github.com/amygdala/tensorflow-workshop/blob/master/workshop_sections/mnist_series/mnist_tflearn.py), and is used with [this README](https://github.com/amygdala/tensorflow-workshop/blob/master/workshop_sections/mnist_series/02_README_mnist_tflearn.md).  It shows how to use TensorFlow's high-level apis, in `contrib.tflearn`, to easily build a classifier with multiple hidden layers.

First, do some imports and set some variables:

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import time

import numpy
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

DATA_DIR = "MNIST_data"
MODEL_DIR = os.path.join("/tmp/tfmodels/mnist_tflearn",
                         str(int(time.time())))
NUM_STEPS = 25000
# read in data, downloading first as necessary
DATA_SETS = input_data.read_data_sets(DATA_DIR)

# comment out for less info during the training runs.
tf.logging.set_verbosity(tf.logging.INFO)


Next, add a function that defines a `DNNClassifier`, and runs its `fit()` method, which will train the model. Note that we didn't need to explicitly define a model graph or a training loop ourselves.

In [ ]:
def define_and_run_dnn_classifier():
    """Run a DNN classifier."""
    feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(
        DATA_SETS.train.images)
    classifier = tf.contrib.learn.DNNClassifier(
        feature_columns=feature_columns, n_classes=10,
        hidden_units=[128, 32],
        # After you've done a training run with optimizer learning rate 0.1,
        # change it to 0.5 and run the training again.  Use TensorBoard to take
        # a look at the difference.  You can see both runs by pointing it to the
        # parent model directory, which by default is:
        #   tensorboard --logdir=/tmp/tfmodels/mnist_tflearn
        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=0.1),
        model_dir=MODEL_DIR
        )
    classifier.fit(DATA_SETS.train.images,
                   DATA_SETS.train.labels.astype(numpy.int64),
                   batch_size=100, max_steps=NUM_STEPS)
    print("Finished running the training via the fit() method")
    return classifier

Then, add a function that runs the classifier's `evaluate()` method.  To do this, it loads the most recent checkpointed model info available. The model checkpoint(s) will be generated during the training process.

In [ ]:
def eval_dnn_classifier(classifier):
    # Evaluate classifier accuracy.
    accuracy_score = classifier.evaluate(
        DATA_SETS.test.images,
        DATA_SETS.test.labels.astype(numpy.int64))['accuracy']
    print('DNN Classifier Accuracy: {0:f}'.format(accuracy_score))

Next, call the function that defines and trains the model. With the default number of steps (25000), it will take a few minutes. Wait for the "finished running..." message to appear.

In [ ]:
print("Running DNN classifier...")
classifier = define_and_run_dnn_classifier()

Now that you've trained the model, evaluate how well it did:

In [ ]:
eval_dnn_classifier(classifier)